Docs: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-mltable?view=azureml-api-2&tabs=cli

if not on Azure ml CLI, run 
```
%pip install -U mltable azureml-dataprep[pandas]
```

In [7]:
%pip install -U mltable azureml-dataprep[pandas]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 44.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: azureml-dataprep-rslex
    Found existing installation: azureml-dataprep-rslex 2.18.3
    Uninstalling azureml-dataprep-rslex-2.18.3:
      Successfully uninstalled azureml-dataprep-rslex-2.18.3
  Attempting uninstall: azureml-dataprep-native
    Found existing installation: azureml-dataprep-native 38.0.0
    Uninstalling azureml-dataprep-native-38.0.0:
      Successfully uninstalled azureml-dataprep-native-38.0.0
  Attempting uninstall: azureml-dataprep
    Found existing installation: azureml-dataprep 4.11.3
    Uninstalling azureml-dataprep-4.11.3:
      Successfully uninstalled azureml-dataprep-4.11.3
  Attempting uninsta

## Author an ML Table file manually from remote data


In [8]:
import mltable

# glob the parquet file paths for years 2015-19, all months.
paths = [
    {
        "pattern": "wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/green/puYear=2015/puMonth=*/*.parquet"
    },
    {
        "pattern": "wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/green/puYear=2016/puMonth=*/*.parquet"
    },
    {
        "pattern": "wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/green/puYear=2017/puMonth=*/*.parquet"
    },
    {
        "pattern": "wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/green/puYear=2018/puMonth=*/*.parquet"
    },
    {
        "pattern": "wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/green/puYear=2019/puMonth=*/*.parquet"
    },
]

# create a table from the parquet paths
tbl = mltable.from_parquet_files(paths)

# table a random sample
tbl = tbl.take_random_sample(probability=0.001, seed=735)

# filter trips with a distance > 0
tbl = tbl.filter("col('tripDistance') > 0")

# Drop columns
tbl = tbl.drop_columns(["puLocationId", "doLocationId", "storeAndFwdFlag"])

# Create two new columns - year and month - where the values are taken from the path
tbl = tbl.extract_columns_from_partition_format("/puYear={year}/puMonth={month}")

# print the first 5 records of the table as a check
tbl.show(5)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,rateCodeID,...,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,year,month
0,2,2015-01-01 21:12:07,2015-01-01 21:15:41,3,0.71,-73.925369,40.761669,-73.923599,40.754658,1,...,0.5,0.5,0.3,1.00,0.0,None,6.80,1,2015,1
1,2,2015-01-01 03:46:04,2015-01-01 04:01:51,1,1.06,-73.964684,40.682896,-73.961937,40.678196,1,...,0.5,0.5,0.3,0.00,0.0,None,7.80,1,2015,1
2,2,2015-01-01 03:55:56,2015-01-01 04:10:34,1,4.90,-73.989822,40.691109,-73.974564,40.646412,1,...,0.5,0.5,0.3,4.38,0.0,None,22.68,1,2015,1
3,2,2015-01-01 03:54:46,2015-01-01 04:14:51,1,3.57,-73.932167,40.707943,-73.963890,40.692127,1,...,0.5,0.5,0.3,0.00,0.0,None,16.80,1,2015,1
4,2,2015-01-01 04:01:51,2015-01-01 04:09:30,5,1.52,-73.889145,40.747181,-73.895729,40.731693,1,...,0.5,0.5,0.3,0.00,0.0,None,8.80,1,2015,1


In [ ]:
# You can load the table into a pandas dataframe
# NOTE: The data is in East US region and the data is large, so this will take several minutes (~7mins)
# to load if you are in a different region.

# df = tbl.to_pandas_dataframe()

In [9]:
# grab the environment variables from the current compute instance 
import os
import json
with open('/config.json', 'r') as f:
  env = json.loads(f.read())
print(env)

{'subscription_id': '1b78932d-85c1-434a-abec-b9aa3cef3c79', 'resource_group': 'azuregigaml', 'workspace_name': 'azuregigaml-ws'}


In [14]:
# save the data loading steps in an MLTable file to a cloud storage
# use an existing datastore
datastore_name = 'demodatastore'
storage_path = f"azureml://subscriptions/{env['subscription_id']}/resourcegroups/{env['resource_group']}/workspaces/{env['workspace_name']}/datastores/{datastore_name}/paths/titanic"
print(f'saving to {storage_path}')

tbl.save(path=storage_path,  overwrite=True)

saving to azureml://subscriptions/1b78932d-85c1-434a-abec-b9aa3cef3c79/resourcegroups/azuregigaml/workspaces/azuregigaml-ws/datastores/demodatastore/paths/titanic


In [15]:
tbl.save()